In [154]:
import os
import pandas as pd
import json
from PIL import Image
import numpy as np
import pickle
import math

a1dir = "../../../scratch/lt2326-h21/a1"
imgs = os.listdir(a1dir+'/images')

In [6]:
def expand_bbox(df):
    """
    Expands the content of each 'adjusted_bbox' cell in dataframe to it's own dataframe.
    Adds file name to each row.
    Returns: dataframe of all bboxes.
    """
    #df = df[:1000] # OBS 
    df = df[['file_name', 'annotations']]
    L = []
    
    for file in df.iterrows():
        for sign in file[1]['annotations']:
            idx = len(sign) * [file[1]['file_name']]
            onesign = pd.DataFrame(sign, index=idx)
            L.append(onesign)
    return pd.concat(L)

In [11]:
def get_img_id_metadata(a1dir):
    df = pd.read_json(a1dir + "/train.jsonl", lines=True)
    imgs = os.listdir(a1dir+'/images')
    print(df.columns)
    df =  df[df['file_name'].isin(imgs)]
    
    maindf = expand_bbox(df)
    
    maindf = maindf[maindf['is_chinese'] == True] # ensures only chinese characters are in
    
    maindf = maindf.reset_index()
    maindf.columns = ['file_name', 'adjusted_bbox', 'attributes', 'is_chinese', 'polygon', 'text']
    #maindf = maindf.set_index(['file_name', maindf.index]) # multiindex
    
    return maindf

In [12]:
mdf = get_img_id_metadata(a1dir)

Index(['annotations', 'file_name', 'height', 'ignore', 'image_id', 'width'], dtype='object')


In [13]:
mdf

,file_name,adjusted_bbox,attributes,is_chinese,polygon,text
0,0000172.jpg,"[140.26028096262758, 897.1957001682758, 22.167...","[distorted, raised]",True,"[[140.26028096262758, 896.7550603352049], [162...",明
1,0000172.jpg,"[162.42785410327272, 898.5416545674744, 23.376...","[distorted, raised]",True,"[[162.42785410327272, 898.0769798344178], [185...",海
2,0000172.jpg,"[185.80456759704398, 899.90363223333, 21.76452...","[distorted, raised]",True,"[[185.80456759704398, 899.4710040335876], [207...",地
3,0000172.jpg,"[207.5690939533138, 901.1694702992428, 20.1523...","[distorted, raised]",True,"[[207.5690939533138, 900.7688886328147], [227....",产
4,0000172.jpg,"[235.37932207521408, 904.5575704933226, 18.943...","[bgcomplex, distorted, raised]",True,"[[235.37932207521408, 904.0339373539472], [254...",易
...,...,...,...,...,...,...
30286,1003106.jpg,"[273.9634547566999, 1209.2381156476695, 18.185...",[distorted],True,"[[273.42684586076234, 1209.2696604538269], [29...",四
30287,1003106.jpg,"[309.1094650877269, 1209.1454775789582, 17.297...",[distorted],True,"[[308.57285619178936, 1209.1754352250573], [32...",六
30288,1003106.jpg,"[325.87076808017883, 1209.0954076389962, 20.17...",[distorted],True,"[[325.3341591842413, 1209.1304987559085], [344...",级
30289,1003106.jpg,"[345.50424799269496, 1209.0483260238314, 17.06...",[distorted],True,"[[344.9676390967574, 1209.07786208054], [361.4...",考


In [22]:
pickle.dump(mdf, open('mdf.p', 'wb'))

In [23]:
mdf = pickle.load(open('mdf.p', 'rb'))

In [80]:
mdf['adjusted_bbox'][0]

[140.26028096262758, 897.1957001682758, 22.167573140645146, 38.36424196832945]

In [14]:
file_names = set(mdf.file_name)

In [84]:
def get_data(filenames, datadir, imgs):
    return {x : np.array(Image.open("{}/{}".format(datadir, x)).convert('RGB')) for x in filenames if x in imgs}

In [85]:
data = get_data(file_names, a1dir+'/images', imgs)

In [86]:
pickle.dump(data, open("data.p", "wb"))

In [120]:
data = pickle.load(open("data.p", "rb"))

In [164]:
data['1003106.jpg']

(2048, 2048, 3)

In [149]:
x, y, w, h = sdfpas.loc[4758]['adjusted_bbox']
print(x, y, w, h)

32.74755123049854 327.3799259999907 59.44940069536657 51.707966193062475


In [169]:
df.annotations[0]

[[{'adjusted_bbox': [140.26028096262758,
    897.1957001682758,
    22.167573140645146,
    38.36424196832945],
   'attributes': ['distorted', 'raised'],
   'is_chinese': True,
   'polygon': [[140.26028096262758, 896.7550603352049],
    [162.42785410327272, 898.0769798344178],
    [162.42785410327272, 935.7929346470926],
    [140.26028096262758, 935.0939571156308]],
   'text': '明'},
  {'adjusted_bbox': [162.42785410327272,
    898.5416545674744,
    23.376713493771263,
    37.74268246537315],
   'attributes': ['distorted', 'raised'],
   'is_chinese': True,
   'polygon': [[162.42785410327272, 898.0769798344178],
    [185.80456759704398, 899.4710040335876],
    [185.80456759704398, 936.5300382257251],
    [162.42785410327272, 935.7929346470926]],
   'text': '海'},
  {'adjusted_bbox': [185.80456759704398,
    899.90363223333,
    21.764526356269812,
    37.08391855844286],
   'attributes': ['distorted', 'raised'],
   'is_chinese': True,
   'polygon': [[185.80456759704398, 899.4710040335876

In [185]:
laks = np.random.rand(4,4,3)

In [180]:
laks

array([[[0.55878611],
        [0.73171942],
        [0.72579614],
        [0.57296535]],

       [[0.2841415 ],
        [0.73952823],
        [0.16292563],
        [0.26447265]],

       [[0.81279148],
        [0.65550882],
        [0.92246155],
        [0.07475837]],

       [[0.41091245],
        [0.57785568],
        [0.37554757],
        [0.85865582]]])

In [191]:
laksy = np.zeros((4, 4, 1))

In [192]:
laksy[0,3,0] = 1

In [171]:
# DU ÄR HÄR SAGAAAAA
# create dict here with structure {filename : binary array of shape [2048, 2048, 1]}

for idx, image_data in enumerate(data.items()):
    fn = image_data[0]
    img = image_data[1]
    mdf_fn = mdf[mdf['file_name'] == fn]['adjusted_bbox']
    # create np.zeros of shape [2048, 2048, 1]
    for i, _ in np.ndenumerate(img):
        imgx, imgy = i[:2]
        for x, y, w, h in mdf_fn:
            xmin = int(math.floor(x))
            xmax = int(math.ceil(x + w))
            ymin = int(math.floor(y))
            ymax = int(math.ceil(y + h))
            if (xmin < imgx < xmax) and (ymin < imgy < ymax):
                # then put a 1 into the all zeros array at the right position ie [imgx, imgy]

NameError: name 'ouihoujn' is not defined

4194304

In [ ]:
for fn, img in data.items():
    